In [1]:
%%writefile example_server.py
from datetime import datetime

from mcp.server.fastmcp import FastMCP

mcp = FastMCP('Test')


@mcp.tool()
def get_current_time() -> str:
    """Get the current time."""
    return f'current_time: {datetime.now().isoformat()}'


@mcp.tool()
def tavily_search(search_query: str) -> str:
    """Performs a web search using the Tavily API."""
    return f"Results for '{search_query}' from Tavily search."


@mcp.tool()
def get_weather(city: str, metric: str = 'celsius') -> str:
    """Get weather info for a city."""
    return f'It is 23 {metric} in {city}'


@mcp.tool()
def get_current_weather(location: str, unit: str = 'celsius', user: dict = None) -> dict:
    """Get the current weather in a location, customized by user details."""
    if not user or 'name' not in user:
        raise ValueError("User details with 'name' field are required.")

    return {'location': location, 'unit': unit, 'user': user, 'weather': f'Sunny, 22 degrees {unit}'}


@mcp.tool()
def get_user_info(user_id: int, special: str = 'none') -> dict:
    """Retrieve user details by ID."""
    return {'user_id': user_id, 'special': special, 'info': {'name': 'John Doe', 'email': 'john.doe@example.com'}}


@mcp.tool()
def yahoo_finance_search(symbol: str) -> dict:
    """Get current stock information for a given symbol."""
    return {'symbol': symbol, 'price': '123.45', 'currency': 'USD'}


@mcp.tool()
def exa_news_search(query: str, answer: bool = False) -> dict:
    """Search news via Exa API."""
    return {'query': query, 'answer_required': answer, 'results': ['News headline 1', 'News headline 2']}


@mcp.tool()
def my_adder_tool(a: int, b: int) -> str:
    """Takes two integers and returns their sum."""
    return f'sum: {a + b}'


if __name__ == '__main__':
    mcp.run(transport='stdio')

Writing example_server.py


In [2]:
import json
from typing import Any, Dict, List

from openai import OpenAI

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

In [ ]:
client = OpenAI(
    base_url='https://api.sambanova.ai/v1/',
    api_key='SAMBANOVA-API-KEY',
)

In [4]:
def function_calling(
    messages: List[Dict[str, Any]],
    client: OpenAI,
    model: str,
    tools: List[Dict[str, Any]] = None,
    tool_choice: str = 'auto',
    parallel_tool_calls: bool = False,
    response_format: Dict[str, Any] = None,
    stream: bool = False,
) -> Any:
    if tools is not None:
        tools_args = {'tools': tools, 'tool_choice': tool_choice, 'parallel_tool_calls': parallel_tool_calls}
    else:
        tools_args = {}

    results = []

    try:
        completion = client.chat.completions.create(
            model=model, messages=messages, stream=stream, response_format=response_format, **tools_args
        )
        if stream:
            for chunk in completion:
                results.append(chunk.choices)
        else:
            if completion and hasattr(completion, 'error'):
                results = f'Error: {completion.error}'
            else:
                results = completion.choices[0].message
    except Exception as e:
        raise e

    return results


def mcp_to_json_schema(mcp_tool: Dict[str, Any]) -> Dict[str, Any]:
    json_schema = {
        'type': 'function',
        'function': {
            'name': mcp_tool['name'],
            'description': mcp_tool['description'],
            'parameters': {'type': 'object', 'properties': {}},
        },
    }

    if 'inputSchema' in mcp_tool and mcp_tool['inputSchema']:
        input_schema = mcp_tool['inputSchema']
        if 'properties' in input_schema:
            for param_name, param_details in input_schema['properties'].items():
                json_schema['function']['parameters']['properties'][param_name] = {
                    'type': param_details['type'],
                    'description': param_details.get('description', ''),
                }

                if 'required' in input_schema and param_name in input_schema['required']:
                    if 'required' not in json_schema['function']['parameters']:
                        json_schema['function']['parameters']['required'] = []
                    json_schema['function']['parameters']['required'].append(param_name)

                if 'default' in param_details:
                    json_schema['function']['parameters']['properties'][param_name]['default'] = param_details[
                        'default'
                    ]

    if '$defs' in mcp_tool:
        json_schema['function']['parameters']['$defs'] = mcp_tool['$defs']

    return json_schema


async def mcp_client(
    available_tools: List[str],
    client: OpenAI,
    model: str,
    messages: List[Dict[str, Any]],
    stream: bool,
    server_params: Dict[str, Any],
) -> Any:
    server_params = StdioServerParameters(**server_params)

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            tools = await session.list_tools()

            tools_schemas = []

            for tool in tools.tools:
                if tool.name in available_tools:
                    tool_dict = json.loads(tool.model_dump_json())
                    final_tool = mcp_to_json_schema(tool_dict)
                    tools_schemas.append(final_tool)
            response = function_calling(messages, client, model, tools_schemas, stream=stream)

            tool_result = await session.call_tool(
                response.tool_calls[0].function.name, arguments=json.loads(response.tool_calls[0].function.arguments)
            )

            messages.append(
                {'role': 'tool', 'tool_call_id': response.tool_calls[0].id, 'content': str(tool_result.content[0].text)}
            )

            final_response = function_calling(messages, client, model)

            return final_response

In [ ]:
custom_server_params = {'command': 'python', 'args': ['example_server.py'], 'env': None}

path = '/Users/user/Downloads'

server_params = {'command': 'npx', 'args': ['-y', '@modelcontextprotocol/server-filesystem', path], 'env': None}

In [ ]:
example = (
    [
        {
            'role': 'system',
            'content': 'You are a helpful assistant. uUse tools available to answer user questions',
        },
        {
            'role': 'user',
            'content': "I am based in Cambridge and need to catch a train soon. What's the current time?",
        },
    ],
    ['get_current_time', 'tavily_search'],
)

example_2 = (
    [
        {
            'role': 'system',
            'content': 'You are a helpful assistant. uUse tools available to answer user questions',
        },
        {
            'role': 'user',
            'content': f'List all the file in {path} folder',
        },
    ],
    ['list_directory'],
)

In [8]:
response = await mcp_client(example[1], client, 'Meta-Llama-3.3-70B-Instruct', example[0], False, custom_server_params)

In [9]:
response

ChatCompletionMessage(content='The current time is 12:54:51. Please check the latest train schedule to ensure you catch your train on time.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

In [10]:
response_2 = await mcp_client(example_2[1], client, 'Meta-Llama-3.3-70B-Instruct', example_2[0], False, server_params)

In [11]:
response_2

ChatCompletionMessage(content="It seems like you're trying to list all the files in the `/Users/luiss/Downloads` folder. The output you provided shows a mix of files and directories. Here's a breakdown:\n\n**Files:**\n\n1. `.DS_Store`\n2. `.localized`\n\n**Directories:**\n\n1. `audio_test`\n2. `custom_data`\n3. `github`\n4. `images_test`\n5. `kubernetes`\n6. `models`\n7. `postman`\n\nPlease note that the output might not be exhaustive, as the command or method used to generate the list might have limitations or filters applied. If you need to list all files and subdirectories recursively, you can use a command like `ls -R /Users/luiss/Downloads` in a terminal or command prompt.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)